In [3]:
import pandas as pd
import json
import os
import re
import numpy as np

### adjust your analysis file and paths

In [5]:
#myfilepath = "enter/path/here"
myfilepath = r"F3+6_neuron and tissue analysis\neurons/"
import sys
sys.path.append(myfilepath)  
import analysis_hist #adjust file here

In [7]:
#open all json result files
#adjust files accordingly 
with open (myfilepath + "F3_W1_M.json") as f:
    results_w1_m = json.load (f) 
    
with open (myfilepath + "F3_W5_M.json") as f:
    results_w5_m = json.load (f)
    
with open (myfilepath + "F6_W1_F.json") as f:
    results_w1_f = json.load (f)   
 
with open (myfilepath + "F6_W5_F.json") as f:
    results_w5_f = json.load (f)

## prepare data, IDs and functions

In [9]:
#set data in groups
w1_m_il = analysis_hist.GroupData (results_w1_m, "IL")
w1_m_cl = analysis_hist.GroupData (results_w1_m, "CL")
w5_m_il = analysis_hist.GroupData (results_w5_m, "IL")
w5_m_cl = analysis_hist.GroupData (results_w5_m, "CL")

w1_f_il = analysis_hist.GroupData (results_w1_f, "IL" )
w1_f_cl = analysis_hist.GroupData (results_w1_f, "CL" )
w5_f_il = analysis_hist.GroupData (results_w5_f, "IL" )
w5_f_cl = analysis_hist.GroupData (results_w5_f, "CL" )


In [11]:
w1_m_nf_sizecounts_cl = np.mean(w1_m_cl.nf_sizecounts_allrats, axis=0)
w1_m_nf_sizecounts_il = np.mean(w1_m_il.nf_sizecounts_allrats, axis=0)
w1_m_ib4_sizecounts_il = np.mean(w1_m_il.ib4_sizecounts_allrats, axis=0)
w1_m_ib4_sizecounts_cl = np.mean(w1_m_cl.ib4_sizecounts_allrats, axis=0)

w5_m_nf_sizecounts_cl = np.mean(w5_m_cl.nf_sizecounts_allrats, axis=0)
w5_m_nf_sizecounts_il = np.mean(w5_m_il.nf_sizecounts_allrats, axis=0)
w5_m_ib4_sizecounts_il = np.mean(w5_m_il.ib4_sizecounts_allrats, axis=0)
w5_m_ib4_sizecounts_cl = np.mean(w5_m_cl.ib4_sizecounts_allrats, axis=0)

In [13]:
w1_f_nf_sizecounts_cl = np.mean(w1_f_cl.nf_sizecounts_allrats, axis=0)
w1_f_nf_sizecounts_il = np.mean(w1_f_il.nf_sizecounts_allrats, axis=0)
w1_f_ib4_sizecounts_il = np.mean(w1_f_il.ib4_sizecounts_allrats, axis=0)
w1_f_ib4_sizecounts_cl = np.mean(w1_f_cl.ib4_sizecounts_allrats, axis=0)

w5_f_nf_sizecounts_cl = np.mean(w5_f_cl.nf_sizecounts_allrats, axis=0)
w5_f_nf_sizecounts_il = np.mean(w5_f_il.nf_sizecounts_allrats, axis=0)
w5_f_ib4_sizecounts_il = np.mean(w5_f_il.ib4_sizecounts_allrats, axis=0)
w5_f_ib4_sizecounts_cl = np.mean(w5_f_cl.ib4_sizecounts_allrats, axis=0)

In [21]:
paths_1wm = w1_m_il.paths
paths_5wm = w5_m_il.paths

paths_1wf = w1_f_il.paths
paths_5wf = w5_f_il.paths

In [23]:
def find_IDs (path): 
    parts = path.split(os.sep)
    rat = next((p for p in parts if re.match(r"^R\w+", p)), None)
    drg = next((p for p in parts if re.match(r"^L\d+", p)), None)
    if rat and drg:
        return f"{rat}_{drg}"
    return None

In [25]:
#for excel export: set rat IDs as they are in the json files
IDs_1wm = [find_IDs(p) for p in paths_1wm if find_IDs(p) is not None]
IDs_5wm = [find_IDs(p) for p in paths_5wm if find_IDs(p) is not None]

IDs_1wf = [find_IDs(p) for p in paths_1wf if find_IDs(p) is not None]
IDs_5wf = [find_IDs(p) for p in paths_5wf if find_IDs(p) is not None]

## define parameters and get files 

In [33]:
def append_dct (IDs_L, gender, condition, param, group_cl, group_il):
    groups = [("CL", group_cl), ("IL", group_il)] 
    for i in IDs_L: 
            dct["ID"].append (i)
            dct["gender"].append (gender)
            dct["condition"].append (condition)
    
    for label, group in groups:
        data = getattr (group, param) 
        for row in data:
            for i, value in enumerate(row):
                bin_key = f"{group.binsizes[i]} {label}"
                dct[bin_key].append(value)

In [37]:
saveto = "Excel files histogram/NF_new"
param = "nf_sizecounts_allrats"

dct = {"ID" : [], "gender" : [], "condition": [] }

groups = [("CL", w1_m_cl), ("IL", w1_m_il)] #für zuordnung zur jeweiligen Seite. Welche gruppe egal, da alle gleich.  
for label, group in groups: #immer bin CL und IL nebeneinander erstellen
    for bin_name in group.binsizes[:-1]:
        dct[f"{bin_name} {label}"] = []

append_dct (IDs_1wm, "M", "1W", param, w1_m_il, w1_m_cl)
append_dct (IDs_5wm, "M", "5W", param, w5_m_il, w5_m_cl)
append_dct (IDs_1wf, "F", "1W", param, w1_f_il, w1_f_cl)
append_dct (IDs_5wf, "F", "5W", param, w5_f_il, w5_f_cl)

df = pd.DataFrame (dct)
df.to_excel (saveto + ".xlsx")